In [ ]:
# -*- coding: utf-8 -*-
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#    http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or
# implied.
# See the License for the specific language governing permissions and
# limitations under the License.
#

### Notes
- *Based* on URL ... cite source(s)

# Differential Privacy

Differential privacy is a way for formalizing privacy using statistics. It provides a way to maximize the accuracy of queries from, for example, databases while minimizing the chances of identifying its entries. Differential privacy ensures that any single individual's data does not significantly influence the overall outcome of the data analysis, making it difficult for attackers to infer anything about an individual even if they have additional knowledge. Here's a simple explanation of how differential privacy works:

1. **Random Noise Addition**: Differential privacy introduces a certain amount of random noise to the results of queries on the dataset. The amount and type of noise depend on the sensitivity of the query (how much the query result could change if any one individual's data were added or removed) and the desired level of privacy, typically parameterized by **ϵ**, known as the privacy budget.

2. **Sensitivity**: This measures how much a query result could change by altering a single individual's data in the dataset. Queries that affect many rows have higher sensitivity and require more noise to be added to maintain the same level of privacy.

3. **Global vs. Local Differential Privacy**: Global differential privacy adds noise to the output of queries over the entire dataset, whereas local differential privacy adds noise to each individual's data before any analysis.

4. **Privacy Budget (ϵ)**: This parameter controls the trade-off between privacy and accuracy. Lower values of **ϵ** provide stronger privacy (more noise) but lower accuracy, while higher values offer better accuracy but weaker privacy.

# 1. Laplace Noise
Laplace noise is drawn from the Laplace distribution, which is characterized by a sharp peak at its mean (which is often zero when used for differential privacy) and exponential decay on either side, resembling a double-sided exponential distribution. The key property of the Laplace distribution is that it adds a level of noise that is proportional to the sensitivity of the function and inversely proportional to the desired privacy level (given by $\epsilon).

The probability density function (pdf) of the Laplace distribution centered at 0 with scale \( b \) is given by:

$ f(x|b) = \frac{1}{2b}\exp\left(-\frac{|x|}{b}\right) $

For differential privacy, $b$ is set to the sensitivity of the query divided by $\epsilon$:

$ b = \frac{\text{sensitivity}}{\epsilon}$

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
plt.figure(figsize=(4,3))

loc, scale = 0., 1.
s = np.random.laplace(loc, scale, 1000)
count, bins, ignored = plt.hist(s, 30, density=True)

x = np.arange(-8., 8., .01)
pdf = np.exp(-abs(x-loc)/scale)/(2.*scale)
plt.plot(x, pdf)

### Simple example

To apply differential privacy to the `cardio_train_m.csv` dataset, we would typically use a differential privacy library such as Google's TensorFlow Privacy, IBM's diffprivlib, or OpenDP's SmartNoise or simply just Python code as follows.

Let's assume we want to release the count of individuals with cardiovascular disease (`cardio` column) with differential privacy applied. We will:

1. Calculate the true count.
2. Determine the sensitivity of the count query (which is 1 for a count query).
3. Choose an **ϵ** value.
4. Add Laplace noise to the true count based on the sensitivity and **ϵ**.
5. Release the noisy count.

Let's start by loading the dataset and calculating the true count of individuals with cardiovascular disease. Then, we'll add Laplace noise to this count.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load the original dataset
original_data = pd.read_csv('cardio_train_m.csv', delimiter=';')

# Calculate the true count of individuals with cardiovascular disease
true_count_cardio = original_data['cardio'].sum()

# Sensitivity for count query is 1
sensitivity = 1

# Choose an epsilon value for the privacy budget, a typical small value for demonstration
epsilon = 0.1

# Generate Laplace noise based on the sensitivity and epsilon
# The scale of the Laplace noise is sensitivity/epsilon
laplace_noise = np.random.laplace(0, sensitivity/epsilon, 1)[0]

# Add Laplace noise to the true count to get the differentially private count
noisy_count_cardio = true_count_cardio + laplace_noise

(true_count_cardio, noisy_count_cardio)


The true count of individuals with cardiovascular disease in the dataset is 34 979.

- After applying Laplace noise with a privacy budget of ϵ=0.1, we obtain a noisy count of approximately 35 021. This noisy count provides differential privacy for the count query, ensuring that no individual's data significantly influences the result, thereby protecting the privacy of the individuals in the dataset.

- The choice of ϵ is critical: a smaller ϵ provides stronger privacy at the cost of accuracy, while a larger ϵ provides more accurate results but weaker privacy. The selection of ϵ typically depends on the privacy needs and the context in which the data is being used.



### More complex example

Continuing with the application of differential privacy, let's consider a more complex query. For instance, we might want to release the average weight of individuals in the dataset in a differentially private manner. The average is a more sensitive query than the count because changing one person's data could potentially change the average more significantly than it could change a count.

To apply differential privacy to the average weight, we'll need to:

1. Calculate the true average weight.
2. Determine the sensitivity of the average query. For weight, we could assume that the weight is bounded within a reasonable range (for example, 30 to 200 kg) to determine the sensitivity.
3. Choose an **ϵ** value.
4. Add Laplace noise to the true average based on the sensitivity and **ϵ**.
5. Release the noisy average.

We will calculate the sensitivity for the average weight assuming that one individual's weight could change the overall sum by at most 200 kg (the upper bound of the weight range). Since the average is the sum divided by the count of individuals, the sensitivity of the average is the maximum change in the sum (200 kg) divided by the count of individuals.


In [ ]:
# Calculate the true average weight
true_avg_weight = original_data['weight'].mean()

# Assuming weight is bounded within 30 to 200 kg, calculate the sensitivity for the average query
# Sensitivity for average = (max value - min value) / number of individuals
max_weight = 200
min_weight = 30
n_individuals = len(original_data)
sensitivity_avg = (max_weight - min_weight) / n_individuals

# Choose an epsilon value for the privacy budget, a typical small value for demonstration
epsilon_avg = 0.1

# Generate Laplace noise based on the sensitivity and epsilon for the average
laplace_noise_avg = np.random.laplace(0, sensitivity_avg/epsilon_avg, 1)[0]

# Add Laplace noise to the true average to get the differentially private average
noisy_avg_weight = true_avg_weight + laplace_noise_avg

(true_avg_weight, noisy_avg_weight)


The true average weight of individuals in the dataset is approximately 74.21 kg.

- After applying Laplace noise with a privacy budget of **ϵ** = 0.1, we obtain a noisy average weight of approximately 74.15 kg. This noisy average provides differential privacy for the average weight query. It ensures that the released average weight is less likely to compromise the privacy of any individual in the dataset.

- Here, we see that the application of noise has only slightly changed the average, indicating that our privacy-preserving release of the average weight still retains a high degree of utility.

- These examples illustrate how differential privacy can be applied to statistical queries to protect individual privacy while still allowing for the release of useful information. The exact parameters for **ϵ** and the sensitivity would depend on the specific privacy guarantees desired and the context of the data's use. It's also important to note that differential privacy provides a probabilistic guarantee of privacy, and the level of noise added should be tailored to the specific needs of the dataset and the queries being performed.

Now let's apply Gaussian noise to the dataset. We will assume that we are releasing the sum of the `weight` column with Gaussian differential privacy. First, we'll calculate the true sum, determine the sensitivity, choose $\epsilon$ and $\delta$, calculate $\sigma$, and then add Gaussian noise to the true sum.

# 2. Gaussian Noise
Gaussian noise, also known as normal noise, is drawn from the Gaussian or normal distribution. Unlike the Laplace distribution, the Gaussian distribution has a "bell curve" shape. It is characterized by its mean (usually zero for differential privacy) and its standard deviation.

The pdf of the Gaussian distribution with mean $\mu$ and standard deviation $\sigma$ is given by:

$
f(x|\mu,\sigma) = \frac{1}{\sigma\sqrt{2\pi}}\exp\left(-\frac{(x-\mu)^2}{2\sigma^2}\right)
$

For differential privacy, the standard deviation \( \sigma \) is set to:

$
\sigma = \frac{\text{sensitivity} \times \sqrt{2 \ln(1.25/\delta)}}{\epsilon}
$

where $\delta$ is another privacy parameter that allows the privacy guarantee to hold with probability $ 1 - \delta$.

In [ ]:
mu, sigma = 0, 0.1 # mean and standard deviation
s = np.random.normal(mu, sigma, 1000)
print('mu=', abs(mu - np.mean(s)), 'sigma=', abs(sigma - np.std(s, ddof=1)))

plt.figure(figsize=(4,3))
count, bins, ignored = plt.hist(s, 30, density=True)
plt.plot(bins, 1/(sigma * np.sqrt(2 * np.pi)) * np.exp( - (bins - mu)**2 / (2 * sigma**2) ),
         linewidth=2, color='r')
plt.show()

In [ ]:
# For Gaussian noise, we need an additional parameter delta, which should be less than the inverse of the dataset size
delta = 1/(n_individuals**2)

# The sensitivity for the sum is the maximum change in the sum if one person's data is changed
# Since we assume the weight to be bounded between 30 and 200, the sensitivity is 200
sensitivity_sum = 200

# Calculate sigma for Gaussian noise based on sensitivity, epsilon, and delta
sigma = (sensitivity_sum * np.sqrt(2 * np.log(1.25/delta))) / epsilon_avg

# Calculate the true sum of weights
true_sum_weight = original_data['weight'].sum()

# Generate Gaussian noise based on the calculated sigma
gaussian_noise = np.random.normal(0, sigma, 1)[0]

# Add Gaussian noise to the true sum to get the differentially private sum
noisy_sum_weight = true_sum_weight + gaussian_noise

(true_sum_weight, noisy_sum_weight)

The true sum of weights in the dataset is approximately 5,194,398.3 kg.

After applying Gaussian noise with the specified parameters \(including the privacy budget $\epsilon = 0.1$ and $\delta$ corresponding to the inverse of the square of the dataset size\), we obtain a noisy sum of weights of approximately 5,204,491.28 kg.

This noisy sum provides differential privacy for the sum query. The Gaussian noise ensures that the released sum of weights is less likely to compromise the privacy of any individual in the dataset, although it is generally less strict than the Laplace mechanism in terms of the probability of privacy guarantee due to the inclusion of $\delta$, allowing for a small probability of the guarantee not holding.

The parameters for $\epsilon$ and $\delta$ should be chosen based on the desired level of privacy guarantee and the context in which the data is being used. The Gaussian mechanism is often used in situations where the query function has higher sensitivity or when a smoother distribution of noise is desired.



# 3. Global Differential Privacy (GDP)

In the Global Differential Privacy model, noise is added to the output of queries performed on the dataset. This model assumes a trusted curator who has access to the raw data and is responsible for adding noise to the query results before sharing them. This curator ensures that the statistical information about the dataset as a whole is preserved while protecting the privacy of individuals.

##### Example for our dataset:
Imagine we want to release the average age of individuals with cardiovascular diseases from our dataset. In a GDP setting, we would:

1. Calculate the true average age from the raw data.
2. Determine the sensitivity of the average age query.
3. Choose an $\epsilon$ value for our privacy budget.
4. Add noise to the true average age (using a mechanism like the Laplace or Gaussian mechanisms we discussed earlier).
5. Release this noisy average age.

The curator (in our case, the data scientist applying the noise) is trusted not to disclose the raw data.

# 4. Local Differential Privacy (LDP)

In the Local Differential Privacy model, noise is added to each individual's data before any analysis or collection takes place. There is no need for a trusted curator because the data is anonymized at the source. This model is more suited for situations where individuals do not trust the data collector or where there is no central entity that can be trusted with the raw data.

##### Example for our dataset:
Suppose each individual's weight is sensitive, and we want to collect this information in a way that protects each individual's privacy. In an LDP setting, we would:

1. Each individual adds noise to their own weight data.
2. These noisy weights are then collected.
3. Any analysis, such as computing the average weight, is performed on the noisy data.

In this case, because the noise is added by the individuals themselves, the noise must be significantly larger to maintain privacy, often resulting in a higher loss of data utility.

# 5. GDP vs LDP: Trade-offs

Differential privacy can be implemented in two different models: Local Differential Privacy (LDP) and Global Differential Privacy (GDP). Each model has its own applications, benefits, and trade-offs.

- **Trust**: GDP requires trust in the data curator, while LDP does not.
- **Data Utility**: GDP often allows for more accurate statistical analyses because noise can be finely tuned to the queries being performed. LDP typically results in a larger loss of data utility because the noise is added individually.
- **Privacy Guarantee**: LDP provides a stronger privacy guarantee since the data is anonymized before any collection or analysis. GDP provides a privacy guarantee for the query results but requires a trusted curator.

For our dataset, if we were to apply LDP, each individual's record would be noised before any aggregation or analysis. For example, each individual could add noise to their `weight` or `age` before it is sent to the researcher. If we were to apply GDP, we would collect all the raw data, and the researcher would add noise to the final computations of statistics like average weight or prevalence of cardiovascular diseases.

Let's illustrate both LDP vs GDP. We'll use the `weight` column for this example.

For LDP, each individual's weight will be noised before any analysis.   
For GDP, we will calculate the sum of the `weight` column and then add noise to this sum.

In LDP, we will add noise directly to each individual's weight. The noise level is typically higher to ensure privacy, given that we do not aggregate data before adding noise.

In [ ]:
# LDP
def apply_ldp(data, epsilon):
    # Assume weights are bounded between 30 and 200 kg for sensitivity calculation
    sensitivity = 200 - 30
    # Each individual adds Laplace noise to their own weight data
    ldp_noise = np.random.laplace(0, sensitivity/epsilon, len(data))
    # Return the noised data
    return data + ldp_noise

# Apply LDP to the weight column
epsilon_ldp = 1  # Setting epsilon for LDP
ldp_noised_weights = apply_ldp(original_data['weight'], epsilon_ldp)

In GDP, we first calculate the sum or average, and then add noise to this aggregate value. The noise level can be lower because we are adding it to an aggregate rather than individual entries.



In [ ]:
# GDP
def apply_gdp_sum(data, epsilon):
    # Calculate the true sum
    true_sum = data.sum()
    # Sensitivity for sum is the maximum change in sum if one person's data is changed
    sensitivity = 200  # Assuming the weight can change by at most 200 kg for one person
    # Add Laplace noise to the sum
    noise = np.random.laplace(0, sensitivity/epsilon, 1)[0]
    # Return the noised sum
    return true_sum + noise

# Apply GDP to the sum of weights
epsilon_gdp = 0.1  # Setting epsilon for GDP
gdp_noised_sum = apply_gdp_sum(original_data['weight'], epsilon_gdp)

In [ ]:
# Define the function for applying LDP to the weight column
def apply_ldp(data, epsilon):
    # Assume weights are bounded between 30 and 200 kg for sensitivity calculation
    sensitivity = 200 - 30
    # Each individual adds Laplace noise to their own weight data
    ldp_noise = np.random.laplace(0, sensitivity/epsilon, len(data))
    # Return the noised data
    return data + ldp_noise

# Define the function for applying GDP to the sum of weights
def apply_gdp_sum(data, epsilon):
    # Calculate the true sum
    true_sum = data.sum()
    # Sensitivity for sum is the maximum change in sum if one person's data is changed
    sensitivity = 200  # Assuming the weight can change by at most 200 kg for one person
    # Add Laplace noise to the sum
    noise = np.random.laplace(0, sensitivity/epsilon, 1)[0]
    # Return the noised sum
    return true_sum + noise

# Apply LDP to the weight column with a chosen epsilon
epsilon_ldp = 1  # Setting epsilon for LDP
ldp_noised_weights = apply_ldp(original_data['weight'], epsilon_ldp)

# Apply GDP to the sum of weights with a chosen epsilon
epsilon_gdp = 0.1  # Setting epsilon for GDP
gdp_noised_sum = apply_gdp_sum(original_data['weight'], epsilon_gdp)

(ldp_noised_weights.head(), gdp_noised_sum)


Here's what the Local Differential Privacy (LDP) and Global Differential Privacy (GDP) applications yielded:

For LDP:
- We added individual noise to the first five weights in the dataset, which resulted in some unrealistic weights (like negative values), indicating the high level of noise that's been added for privacy. This is an artifact of not bounding the noise properly, which should be done in a real-world application to prevent such anomalies.

For GDP:
- We added noise to the sum of the `weight` column across the entire dataset, which resulted in a noisy sum of approximately 5,198,555.45 kg. This is much closer to the true sum we calculated earlier, indicating a lower level of noise added for privacy, which is a characteristic of the global differential privacy model.

The choice of $\epsilon$ is crucial in both cases. A lower $\epsilon$ increases privacy but adds more noise, potentially leading to less useful data. The $\epsilon$ values should be chosen based on the acceptable trade-off between privacy and utility for the particular context in which the data is used.

In [ ]:
# Since the provided GDP method applies noise to the sum of weights, we cannot create a histogram for it.
# However, we can create a histogram for the original and LDP-applied weights.

# Preparing the figure for histograms
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10, 4))
fig.suptitle('Comparative Histograms of Weight Distribution', fontsize=16)

# Histogram for original weight distribution
sns.histplot(original_data['weight'], bins=30, kde=True, ax=axes[0], color='blue')
axes[0].set_title('Original Weight Distribution')
axes[0].set_xlabel('Weight (kg)')
axes[0].set_ylabel('Frequency')

# Histogram for weight distribution after applying LDP
sns.histplot(ldp_noised_weights, bins=30, kde=True, ax=axes[1], color='green')
axes[1].set_title('Weight Distribution after LDP')
axes[1].set_xlabel('Weight (kg)')
axes[1].set_ylabel('Frequency')

plt.tight_layout(rect=[0, 0.03, 1, 0.95])  # Adjust layout to fit the title
plt.show()


In [ ]:
# Preparing to visualize the GDP result
original_sum = original_data['weight'].sum()
gdp_comparison = pd.DataFrame({'Sum Type': ['Original Sum', 'Noised Sum'],
                               'Total Weight': [original_sum, gdp_noised_sum]})

# Visualization of the GDP result
plt.figure(figsize=(6, 4))
sns.barplot(x='Sum Type', y='Total Weight', data=gdp_comparison)
plt.title('Comparison of Original and Noised Sum of Weights (GDP)')
plt.show()